# Módulo 2: Scraping con Selenium
## LATAM Airlines
<a href="https://www.latam.com/es_ar/"><img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png"  width="420"></img></a>
<br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:
- Precio(s) disponibles
- Horas de salida y llegada (duración)
- Información de las escalas

**¡Empecemos!**
Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto

## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo `webdriver`.

In [1]:
from selenium import webdriver

# #importampos libreria para cargar el driver automaticamente de firefox
# from webdriver_manager.firefox import GeckoDriverManager

#importampos libreria para cargar el driver manualemente de google chrome
from selenium.webdriver.chrome.options import Options
import time

#pagina web que vamos a hacer el scraping
url='https://www.latamairlines.com/py/es/ofertas-vuelos?origin=ASU&outbound=2023-08-01T12%3A00%3A00.000Z&destination=BCN&inbound=null&adt=1&chd=0&inf=0&trip=OW&cabin=Economy&redemption=false&sort=RECOMMENDED'

## Demora Dinamica
<p>Para evitar fallos de carga de la página y de mensajes emergente que no dejen realizar el scraper, se implementan demoras dinámicas, estas a diferencia de la demora estática brindan un mejor rendimiento. Ya que no tienen que esperar el tiempo estático sino que esperan a que se cargue la página y si tarda menos del tiempo estipulado, empieza a hacer el scraper.</p>

 Lo mejor sería esperar a que la página termine de cargar y luego recuperar los elementos. para ello importamos varias librerias

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''

    tarifas= vuelo.find_elements('xpath','.//ol[@class="sc-buGlAa jhwXGF"]/li')
    precios=[]

    for tarifa in tarifas:
        #buscamos en cada pocision de la tarifa los siquientes elementos
        nombre = tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[1]/span[@class="sc-fhiYOA iwcbaW"]').text
        moneda= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[contains(@class,"currency")]').text
        valor= tarifa.find_element('xpath','.//div[@class="sc-gGsJSs dhstcp"]/div[3]//span[@class="sc-ckYZGd grNCid"]').text
        
        #guardo los valores que obtengo en un diccionario
        dict_tarifa={nombre:{'moneda':moneda,'valor':valor}}
        
        #guardamos nuestro diccionario con los datos de tarifa a nuestra lista de precio
        precios.append(dict_tarifa)
    return precios

In [4]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de 
    las escalas de cada vuelo
    '''
    
    segmentos= vuelo.find_elements('xpath','//article[@class="sc-lffWgi vYKpy"]//section[@class="sc-fGSyRc fCuylQ"]')
    datos_escalas = []
    duracion_escalas_dic = {}
    for segmento in segmentos:
        # espera inteligente para encontrar un elemento y guardar
        wait_seg = WebDriverWait(segmento, 15)

        # origen
        #origen_segmento=segmento.find_element(By.XPATH,'.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[1]').text
        
        #origen demora - inteligente
        origen_segmento = wait_seg.until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[last()-1]'))).text

        # hora de salida
        #hora_salida=segmento.find_element(By.XPATH,'.//div[@class="sc-jFpLkX jAGOAr"]/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[@class="time"]').text
        
        # hora de salida - demora inteligente
        hora_salida = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-fguZLD kepXur"]/div[@class="iataCode"]/span[2]'))).text

        # destino
        #destino_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[1]').text
        
        # desitno - demora inteligente
        destino_segmento = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[1]'))).text

        # Hora de llegada
        #hora_llegada=segmento.find_element('xpath','.//div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[2]').text

        #Hora de llegada - demora inteligente
        hora_llegada = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-eCXBzT goeYBu"]/div[@class="iataCode"]/span[2]'))).text

        # Duracion del vuelo
        #duracion_segmento=segmento.find_element('xpath','.//div[@class="sc-jFpLkX jAGOAr"]//div[@class="sc-ewMkZo hQNSAX"]/span[2]').text

        #Duracion del vuelo - demora inteligente
        duracion_segmento = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-ewMkZo hQNSAX"]/span[2]'))).text

        # Numero de vuelo
        #numero_vuelo_segmento=segmento.find_element('xpath','.//div[@class="sc-dzQEYZ dslPlz airline-wrapper"]').text

        # Numero de vuelo - demora inteligente
        numero_vuelo_segmento = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-dzQEYZ dslPlz airline-wrapper"]'))).text

        # Modelo de avion
        #modelo_avion_segmento=segmento.find_element('xpath','.//div[@class="sc-sVRsr eXYUTi"]//span[@class="airplane-code"]').text

        # Modelo de avion - demora inteligente 
        modelo_avion_segmento = WebDriverWait(segmento, 15).until(EC.presence_of_element_located(
            (By.XPATH, './/div[@class="sc-sVRsr eXYUTi"]//span[@class="airplane-code"]'))).text

        # Armo un diccionario para almacenar los datos
        datos_escalas_dict = {
            'origen': origen_segmento,
            'hora_salida': hora_salida,
            'destino': destino_segmento,
            'hora_llega': hora_llegada,
            'duracion': duracion_segmento,
            'numero_vuelo': numero_vuelo_segmento,
            'modelo_avion': modelo_avion_segmento
        }
        datos_escalas.append(datos_escalas_dict)

    # seleccionamos los segmentos(section) del modal  pertencientes a las escalas
    escalas_vuelo = vuelo.find_elements(
        'xpath', '//section[@class="sc-kiXyGy sc-eZXMBi dKgCnQ connectionInfo"]')
    
    #recorremos cada segmento de escala para obtener su informacion
    for num_escala, escala in enumerate(escalas_vuelo):
        escala = escala
        # conexion de la escala
        escala_vuelo = escala.find_element(
            'xpath', './/div[@class="sc-ekQYnd cByWfv"]//span[@class="connection-text"]').text
        # duracion de la escala
        duracion_escala_vuelo = escala.find_element(
            'xpath', './/div[@class="sc-ekQYnd cByWfv"]//span[@class="time"]').text
        # guardamos en un diccionario la escala actual
        duracion_escalas = {f'Escala {num_escala+1}': escala_vuelo,
                            f'Duracion Escala {num_escala+1}': duracion_escala_vuelo}
        # actualizamos nuestro dicionario de escalas totales
        duracion_escalas_dic.update(duracion_escalas)

    # agregamos nuestras escalas totales en nuestra lista de datos
    datos_escalas.append({'duracion_escalas': duracion_escalas_dic})
    return datos_escalas

In [5]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salida y llegada de cada
    vuelo, incluyendo la duración. 
    Nota: la duración del vuelo no es la hora de llegada - la hora de salida porque
    puede haber diferencia de horarios entre el origen y el destino.
    '''
    #hora de salida
    hora_salida=vuelo.find_element('xpath','.//div[@class="sc-klSiHT hjzFuR flight-information"]/span[1]').text
    #hora de llegada
    hora_llegada=vuelo.find_element('xpath','.//div[3]/span[1]').text.replace('\n+1','')
    # Duracion del vuelo
    duracion_vuelo= vuelo.find_element('xpath','.//div[2]/span[2]').text

    tiempos_vuelo_dict={
        'hora_salida':hora_salida,
        'hora_llegada':hora_llegada,
        'duracion_vuelo':duracion_vuelo
    }

    return tiempos_vuelo_dict

In [6]:
def obtener_info(driver):
    #Usaremos el Xpath para obtener la lista de vuelos
    # Sin demora inteligente
    #vuelos = driver.find_elements('xpath','//ol/li[@class="sc-bvTASY cfqKKq"]')
    
    # implementamos demora inteligente para que cargen primero todos los elementos y luego continue el codigo
    wait= WebDriverWait(driver, 10)
    vuelos = wait.until(EC.presence_of_all_elements_located((By.XPATH,'//ol/li[@class="sc-bvTASY cfqKKq"]')))
    
    print(f'Se encontaron {len(vuelos)} vuelos.')
    print(f'Iniciando Scraping...')
    info=[]
    contador=0
    for vuelo in vuelos:
        #contador < 10 -> para hacer pruebas hasta el vuelo numero 10
        if contador<20:
            contador+=1
            print(f'Analizando {contador} de {len(vuelos)} vuelos')

            #obtener los tiempos generales de cada vuelo
            tiempos=obtener_tiempos(vuelo)
            print(f'{tiempos}\n')
            
            #clickeamos sobe el link escalas para que se habra el modal
            #vuelo.find_element('xpath','.//div[@class="sc-iKiVwC fbWfQZ"]//a').click()
            
            #le damos tiempo a que cargue el modal con todos sus elementos para luego poder analizar
            # time.sleep(10)
            WebDriverWait(vuelo, 10).until(EC.presence_of_element_located((By.XPATH, './/div[@class="sc-iKiVwC fbWfQZ"]//a'))).click()
            
            # guardamos nuestro elemento que contiene todas las escales para hacer scroll hasta el ultimo elemento
            element= vuelo.find_element('xpath','//div[@class="MuiDialogContent-root sc-bZQynM hnBdls"]')
            verical_ordinate = 100
            for i in range(0, 10):
                #print(verical_ordinate)
                driver.execute_script("arguments[0].scrollTop = arguments[1]", element, verical_ordinate)
                verical_ordinate += 100
                time.sleep(1)

            print('Termino de cargar las escalas')

            #guardamos las escalas y llama a la funcion -> obtener escalas
            escalas=obtener_datos_escalas(vuelo)
            print(f'{escalas}\n')
            
            #cerramos el modal de las escalas
            driver.find_element('xpath','//div[@class="MuiDialog-container MuiDialog-scrollPaper"]').click()

            #clickeamos el vuelo para ver los precios implementando demora dinamica
            vuelo.find_element('xpath','.//div[@class="sc-cIbcTr iuAYhf"]').click()
            WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.XPATH,'.//ol[@class="sc-buGlAa jhwXGF"]/li')))
            
            print('La tabla de precios terminó de cargar')
            
            #guardamos los precios y llama a la funcion -> obtener precios
            precios=obtener_precios(vuelo)
            print(f'{precios}\n*********************************************\n')
            
            #click nuevamente para cerrar los precios que se desplegaron
            vuelo.find_element('xpath','.//div[@id="undefined--button-wrapper"]/button').click()

            # guardamos toda la informacion recogida en una lista
            info.append({'precios':precios,'tiempos':tiempos,'escalas':escalas})

    return info

In [7]:
#cargar drive automaticamente utilizando webdriver
# options = webdriver.FirefoxOptions()

#cargar drive automaticamente utilizando webdriver
#driver = webdriver.Firefox(executable_path=GeckoDriverManager().install(), options=options)

#carga driver manualmente
options= Options()

# Podemos agregarle opciones al driver para utilizar los distintos modos del navegador
options.add_argument('--incognito')

#carga driver manualmente
driver = webdriver.Chrome(options=options)
driver.maximize_window()
driver.get(url)

#si aparece un mensaje popups apenas inicie la web entonces lo cerramos
def close_dialog():
    '''
    Función que cierra los dialog o popups de la página
    '''
    dialogs = driver.find_elements(By.XPATH, '//div[contains(@class,"MuiDialog-container")]')
    if len(dialogs):
        dialogs[0].find_element(By.XPATH, './/button[contains(@class,"Dialog__CloseButton")]').click()

close_dialog()

#introducir demora/delay o tiempo maximo de espera
delay=15

try:
    # introducir demora inteligente para que carguen toda la lista de vuelos
    WebDriverWait(driver, delay ).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@id="WrapperBodyFlights"]//ol/li[@class="sc-bvTASY cfqKKq"]')))
    print('La página terminó de cargar')
    obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
finally:
    #cerrar el navegador
    driver.close()

La página terminó de cargar
Se encontaron 27 vuelos.
Iniciando Scraping...
Analizando 1 de 27 vuelos
{'hora_salida': '6:55 a. m.', 'hora_llegada': '5:20 p. m.', 'duracion_vuelo': '28 h 25 min'}

Termino de cargar las escalas
[{'origen': 'ASU', 'hora_salida': '6:55 a. m.', 'destino': 'LIM', 'hora_llega': '10:00 a. m.', 'duracion': '4 h 5 min'}, {'origen': 'LIM', 'hora_salida': '8:20 p. m.', 'destino': 'MAD', 'hora_llega': '2:35 p. m.', 'duracion': '11 h 15 min'}, {'origen': 'MAD', 'hora_salida': '4:00 p. m.', 'destino': 'BCN', 'hora_llega': '5:20 p. m.', 'duracion': '1 h 20 min'}, {'duracion_escalas': {'Escala 1': 'Conexión Lima', 'Duracion Escala 1': '10 h 20 min', 'Escala 2': 'Conexión Madrid', 'Duracion Escala 2': '1 h 25 min'}}]

La tabla de precios terminó de cargar
[{'plus': {'moneda': 'USD', 'valor': '1,341.20'}}, {'top': {'moneda': 'USD', 'valor': '3,153.20'}}]
*********************************************

Analizando 2 de 27 vuelos
{'hora_salida': '6:55 a. m.', 'hora_llegada': 